# core

> Utility for automating backups of a specific file or directory

In [ ]:
#| default_exp core

# Automated Backups

We want a script to back up a specific file/folder over different intervals. Specifically, it should

- Copy to some destination dir every hour (e.g. a different drive)
- Keep the last 5, and one every day, week and month (for example)

We can then rsync the destination dir to keep a remote backup.

In [ ]:
#|export
import shutil, os, time, pprint, logging
from pathlib import Path
from fastcore.script import call_parse, bool_arg
from fastcore.xtras import globtastic
from datetime import datetime, timedelta

In [ ]:
!mkdir -p demo_src
!mkdir -p demo_dst
!rm -rf demo_dst/*
!rm -rf demo_src/*

In [ ]:
!echo "content" > "demo_src/test_text.txt"
!echo "## content" > "demo_src/test_two.md"

## The core functionality

The plan has two main steps:

- Create a new backup
- Clean up any old backups that are no longer needed.


For step 1 we want to go file by file in case of errors, and support a matching pattern for what to include. So, take 2:

In [ ]:
globtastic("demo_src", file_glob="*.md") # Finding files

(#1) ['demo_src/test_two.md']

In [ ]:
globtastic("demo_src")

(#2) ['demo_src/test_text.txt','demo_src/test_two.md']

In [ ]:
#| export
def create_backup(src, dest_dir, dry_run=False,
                 recursive:bool=True, symlinks:bool=True, 
                 file_glob:str=None, file_re:str=None,
                 folder_re:str=None, skip_file_glob:str=None,
                 skip_file_re:str=None, skip_folder_re:str=None):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    src_path = Path(src)
    dest_path = Path(dest_dir) / timestamp
    if not dry_run: dest_path.mkdir(parents=True, exist_ok=True)
    
    if src_path.is_file():
        files_to_copy = [src_path]
    else:
        files_to_copy = globtastic(src_path,recursive=recursive, symlinks=symlinks,
                                   file_glob=file_glob, file_re=file_re,
                                   folder_re=folder_re, skip_file_glob=skip_file_glob,
                                   skip_file_re=skip_file_re, skip_folder_re=skip_folder_re)
        files_to_copy = [Path(f) for f in files_to_copy]
    
    for file in files_to_copy:
        if file.is_file():
            rel_path = file.relative_to(src_path)
            dest_file = dest_path / rel_path
            if not dry_run: dest_file.parent.mkdir(parents=True, exist_ok=True)
            try:
                if dry_run: print(f'Copy from {file} to {dest_file}')
                else: shutil.copy2(file, dest_file)
            except Exception as e:
                logging.warning(f"Failed to copy {file}: {e}")

In [ ]:
create_backup('demo_src', 'demo_dst')
!ls demo_dst

20241127_151709


In [ ]:
!ls demo_dst/20241127_151709

test_text.txt  test_two.md


In [ ]:
# Test single file
create_backup('demo_src/test_text.txt', 'demo_dst', dry_run=True)

Copy from demo_src/test_text.txt to demo_dst/20241127_151712


In [ ]:
# Test pattern
create_backup('demo_src', 'demo_dst', file_glob='*.md', dry_run=True)

Copy from demo_src/test_two.md to demo_dst/20241127_151721/test_two.md


In [ ]:
# Test skip_pattern
create_backup('demo_src', 'demo_dst', skip_file_glob='*.md', dry_run=True)

Copy from demo_src/test_text.txt to demo_dst/20241127_151737/test_text.txt


The harder part is the cleanup. Let's start by generating some dates to test with.

In [ ]:
def generate_test_dates(num_dates, base_date):
    return [(base_date + timedelta(hours=i)).strftime("%Y%m%d_%H%M%S") for i in range(num_dates)]
test_dates = generate_test_dates(2400, datetime.now() - timedelta(days=100))
print(test_dates[:5], test_dates[-5:])

['20240819_151740', '20240819_161740', '20240819_171740', '20240819_181740', '20240819_191740'] ['20241127_101740', '20241127_111740', '20241127_121740', '20241127_131740', '20241127_141740']


In [ ]:
# Can I get all dates < 2 months old?
[d for d in test_dates if (datetime.now() - datetime.strptime(d, '%Y%m%d_%H%M%S')).days < 60][:3]

['20240928_161740', '20240928_171740', '20240928_181740']

Now we want to grab the most recent 5, and then the oldest below some threshold.

In [ ]:
#| export 
def clean_dates(dates, now=None, max_ages=(2, 14, 60)):
    now = now or datetime.now()
    clean = []
    dates.sort()
    
    for max_age in max_ages:
        lt_max = [d for d in dates if (now - datetime.strptime(d, '%Y%m%d_%H%M%S')).days < max_age]
        if lt_max: clean.append(lt_max[0])

    clean.extend(dates[-5:])  # Keep the newest 5
    return sorted(set(clean))  # Remove duplicates and sort

In [ ]:
clean_dates(test_dates)

['20240928_161740',
 '20241113_161740',
 '20241125_161740',
 '20241127_101740',
 '20241127_111740',
 '20241127_121740',
 '20241127_131740',
 '20241127_141740']

Now we want code that starts with the same test dates etc as above, but then simulates time passing by adding an hour to 'now' and a date to test dates every step then printing out a (prettified) version of clean_dates to check it's doing as I expect over a simulated month.

In [ ]:
# # Initialize
# now = datetime.now()
# test_dates = generate_test_dates(2400, now - timedelta(days=100))

# # Simulate time passing
# for _ in range(30 * 24):  # Simulate a month (30 days * 24 hours)
#     now += timedelta(hours=1)
#     test_dates.append(now.strftime("%Y%m%d_%H%M%S"))
#     test_dates = clean_dates(test_dates, now)  # Clean up old dates
#     if _ % 24 == 0:  # Print once a day
#         print(f"\nDay {_ // 24 + 1}:")
#         pprint.pprint(test_dates)

NB: Yay, it looks to be doing mostly what I want! I can collapse the output, if you're viewing this in a notebook my apologies :)

## Turning it into a script

Now that those two pieces of functionality seem to be working, we can wrap this up as a script using fastcore's call_parse, have it run the backup, clean up old files, and log any errors or messages to backup.log

In [ ]:
#|export
@call_parse
def run_backup(
    src:str, # The source to be backed up
    dest:str, # The destination directory
    max_ages:str="2,14,60", # The max age(s) in days for the different backups
    log_file:str='backup.log',
    dry_run:bool_arg=False, # Dry run?
    recursive:bool_arg=True, 
    symlinks:bool_arg=True, 
    file_glob:str=None, 
    file_re:str=None,
    folder_re:str=None, 
    skip_file_glob:str=None,
    skip_file_re:str=None, 
    skip_folder_re:str=None
):
    "Run backup and cleanup old files. Takes globtastic args."
    
    # Set up logging
    logging.basicConfig(filename=log_file, level=logging.DEBUG,
                        format='%(asctime)s - %(levelname)s - %(message)s')
    try:
        create_backup(src, dest, dry_run=dry_run,
                     recursive=recursive, symlinks=symlinks,
                     file_glob=file_glob, file_re=file_re,
                     folder_re=folder_re, skip_file_glob=skip_file_glob,
                     skip_file_re=skip_file_re, skip_folder_re=skip_folder_re)
        logging.info(f"Backup created: {src} -> {dest}")
    except Exception as e:
        logging.error(f"Backup failed: {str(e)}", exc_info=True)
    finally:
        max_ages = [int(age.strip()) for age in max_ages.split(',')]
        backups = [d.name for d in Path(dest).iterdir() if d.is_dir()]
        to_keep = clean_dates(backups, max_ages=max_ages)
        for backup in backups:
            if backup not in to_keep:
                try:
                    if dry_run:print('Remove', Path(dest) / backup)
                    else: shutil.rmtree(Path(dest) / backup)
                    logging.info(f"Removed old backup: {backup}")
                except Exception as e:
                    logging.error(f"Removing old backup failed: {str(e)}", exc_info=True)

In [ ]:
!ls demo_src

test_text.txt  test_two.md


Testing a directory:

In [ ]:
!rm -r demo_dst/*

In [ ]:
run_backup('demo_src', 'demo_dst')
!ls demo_dst

20241127_151747


In [ ]:
!ls demo_dst/20241127_151747

test_text.txt  test_two.md


Testing a pattern

In [ ]:
!rm -r demo_dst/*

In [ ]:
run_backup('demo_src', 'demo_dst', skip_file_glob="*.md", dry_run=True)

Copy from demo_src/test_text.txt to demo_dst/20241127_151801/test_text.txt
